In [1]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1647803027404_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Transfromar dados em parquet

In [2]:
rais = (
    spark.read
        .csv("s3://datalake-igti-staging-579937625472/raw-data/rais/", inferSchema=True, header=True, sep=';', encoding="latin1")
)
rais.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Bairros SP: string (nullable = true)
 |-- Bairros Fortaleza: string (nullable = true)
 |-- Bairros RJ: string (nullable = true)
 |-- Causa Afastamento 1: integer (nullable = true)
 |-- Causa Afastamento 2: integer (nullable = true)
 |-- Causa Afastamento 3: integer (nullable = true)
 |-- Motivo Desligamento: integer (nullable = true)
 |-- CBO Ocupação 2002: string (nullable = true)
 |-- CNAE 2.0 Classe: integer (nullable = true)
 |-- CNAE 95 Classe: integer (nullable = true)
 |-- Distritos SP: string (nullable = true)
 |-- Vínculo Ativo 31/12: integer (nullable = true)
 |-- Faixa Etária: integer (nullable = true)
 |-- Faixa Hora Contrat: integer (nullable = true)
 |-- Faixa Remun Dezem (SM): integer (nullable = true)
 |-- Faixa Remun Média (SM): integer (nullable = true)
 |-- Faixa Tempo Emprego: integer (nullable = true)
 |-- Escolaridade após 2005: double (nullable = true)
 |-- Qtd Hora Contr: double (nullable = true)
 |-- Idade: double (nullable = true)
 |-- Ind CEI Vincul

## Correção nomes das colunas

In [3]:
sc.install_pypi_package("unidecode")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/f9/5b/7603add7f192252916b85927263b598c74585f82389e6e42318a6278159b/Unidecode-1.3.4-py3-none-any.whl

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [5]:
import unidecode
import re

def limpa_nome(nome):
    novo_nome = re.sub("\s", "_", unidecode.unidecode(nome))
    
    return re.sub("[()/]", "", novo_nome.lower())

for index, colname in enumerate(rais.columns):
    rais = (
        rais.withColumnRenamed(rais.columns[index], limpa_nome(colname))
    )
rais.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- bairros_sp: string (nullable = true)
 |-- bairros_fortaleza: string (nullable = true)
 |-- bairros_rj: string (nullable = true)
 |-- causa_afastamento_1: integer (nullable = true)
 |-- causa_afastamento_2: integer (nullable = true)
 |-- causa_afastamento_3: integer (nullable = true)
 |-- motivo_desligamento: integer (nullable = true)
 |-- cbo_ocupacao_2002: string (nullable = true)
 |-- cnae_2.0_classe: integer (nullable = true)
 |-- cnae_95_classe: integer (nullable = true)
 |-- distritos_sp: string (nullable = true)
 |-- vinculo_ativo_3112: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)
 |-- faixa_hora_contrat: integer (nullable = true)
 |-- faixa_remun_dezem_sm: integer (nullable = true)
 |-- faixa_remun_media_sm: integer (nullable = true)
 |-- faixa_tempo_emprego: integer (nullable = true)
 |-- escolaridade_apos_2005: double (nullable = true)
 |-- qtd_hora_contr: double (nullable = true)
 |-- idade: double (nullable = true)
 |-- ind_cei_vinculado: 

In [5]:
rais.select('ibge_subsetor').limit(10).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+
|ibge_subsetor|
+-------------+
|           25|
|           19|
|            7|
|           13|
|           10|
|           19|
|           19|
|            2|
|           21|
|            2|
+-------------+

In [6]:
rais.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

241

## Gerando coluna uf

In [6]:
# Para que o comando explain() funcione
from py4j.java_gateway import java_import
java_import(spark._sc._jvm, "org.apache.spark.sql.api.python.*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
rais = rais.withColumn("uf", f.col("municipio").cast('string').substr(1,2).cast('int'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Corrigindo tipos das variáveis de renda e mês de desligamento

In [7]:
rais.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- bairros_sp: string (nullable = true)
 |-- bairros_fortaleza: string (nullable = true)
 |-- bairros_rj: string (nullable = true)
 |-- causa_afastamento_1: integer (nullable = true)
 |-- causa_afastamento_2: integer (nullable = true)
 |-- causa_afastamento_3: integer (nullable = true)
 |-- motivo_desligamento: integer (nullable = true)
 |-- cbo_ocupacao_2002: string (nullable = true)
 |-- cnae_2.0_classe: integer (nullable = true)
 |-- cnae_95_classe: integer (nullable = true)
 |-- distritos_sp: string (nullable = true)
 |-- vinculo_ativo_31/12: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)
 |-- faixa_hora_contrat: integer (nullable = true)
 |-- faixa_remun_dezem_(sm): integer (nullable = true)
 |-- faixa_remun_media_(sm): integer (nullable = true)
 |-- faixa_tempo_emprego: integer (nullable = true)
 |-- escolaridade_apos_2005: double (nullable = true)
 |-- qtd_hora_contr: double (nullable = true)
 |-- idade: double (nullable = true)
 |-- ind_cei_vincul

In [10]:
rais.select(
    'mes_desligamento'
).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|mes_desligamento|
+----------------+
|              06|
|              03|
|              10|
|              07|
|              04|
|              01|
|              03|
|              01|
|              04|
|              05|
+----------------+
only showing top 10 rows

In [8]:
rais = (
    rais
    .withColumn("mes_desligamento", f.col('mes_desligamento').cast('int'))
    .withColumn("vl_remun_dezembro_nom", f.regexp_replace("vl_remun_dezembro_nom", ',', '.').cast('double'))
    .withColumn("vl_remun_dezembro_sm", f.regexp_replace("vl_remun_dezembro_sm", ',', '.').cast('double'))
    .withColumn("vl_remun_media_nom", f.regexp_replace("vl_remun_media_nom", ',', '.').cast('double'))
    .withColumn("vl_remun_media_sm", f.regexp_replace("vl_remun_media_sm", ',', '.').cast('double'))
    .withColumn("vl_rem_janeiro_sc", f.regexp_replace("vl_rem_janeiro_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_fevereiro_sc", f.regexp_replace("vl_rem_fevereiro_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_marco_sc", f.regexp_replace("vl_rem_marco_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_abril_sc", f.regexp_replace("vl_rem_abril_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_maio_sc", f.regexp_replace("vl_rem_maio_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_junho_sc", f.regexp_replace("vl_rem_junho_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_julho_sc", f.regexp_replace("vl_rem_julho_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_agosto_sc", f.regexp_replace("vl_rem_agosto_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_setembro_sc", f.regexp_replace("vl_rem_setembro_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_outubro_sc", f.regexp_replace("vl_rem_outubro_sc", ',', '.').cast('double'))
    .withColumn("vl_rem_novembro_sc", f.regexp_replace("vl_rem_novembro_sc", ',', '.').cast('double'))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
rais.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- bairros_sp: string (nullable = true)
 |-- bairros_fortaleza: string (nullable = true)
 |-- bairros_rj: string (nullable = true)
 |-- causa_afastamento_1: integer (nullable = true)
 |-- causa_afastamento_2: integer (nullable = true)
 |-- causa_afastamento_3: integer (nullable = true)
 |-- motivo_desligamento: integer (nullable = true)
 |-- cbo_ocupacao_2002: string (nullable = true)
 |-- cnae_2.0_classe: integer (nullable = true)
 |-- cnae_95_classe: integer (nullable = true)
 |-- distritos_sp: string (nullable = true)
 |-- vinculo_ativo_3112: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)
 |-- faixa_hora_contrat: integer (nullable = true)
 |-- faixa_remun_dezem_sm: integer (nullable = true)
 |-- faixa_remun_media_sm: integer (nullable = true)
 |-- faixa_tempo_emprego: integer (nullable = true)
 |-- escolaridade_apos_2005: double (nullable = true)
 |-- qtd_hora_contr: double (nullable = true)
 |-- idade: double (nullable = true)
 |-- ind_cei_vinculado: 

## Escrevendo parquet

In [11]:
(
    rais
    .coalesce(50)
    .write.mode('overwrite')
    .partitionBy('year', 'uf')
    .format('parquet')
    .save('s3://datalake-igti-staging-579937625472/consume-zone/rais/')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

An error was encountered:
Session 2 did not reach idle status in time. Current status is busy.
